In [1]:
!pip  install pygame 

In [ ]:
import pygame
import os

# Screen dimensions
WIDTH, HEIGHT = 800, 800

# Board dimensions
ROWS, COLS = 8, 8
SQSIZE = WIDTH // COLS

class Main:
    def __init__(self):
        pygame.init()
        self.screen = pygame.display.set_mode((WIDTH, HEIGHT))
        pygame.display.set_caption('Chess')
        self.game = Game()

    def mainloop(self):
        screen, game = self.screen, self.game
        dragger, board = game.dragger, game.board
        running = True
        
        while running:
            game.show_bg(screen)
            game.show_last_move(screen)
            game.show_moves(screen)
            game.show_pieces(screen)
            game.show_hover(screen)

            if dragger.dragging:
                dragger.update_blit(screen)
            
            for event in pygame.event.get():
                if event.type == pygame.MOUSEBUTTONDOWN:  # Click event 
                    dragger.update_mouse(event.pos)
                    clicked_row = dragger.mouseY // SQSIZE
                    clicked_col = dragger.mouseX // SQSIZE
                    
                    # Checking if clicked square has a piece
                    if board.squares[clicked_row][clicked_col].has_piece():
                        piece = board.squares[clicked_row][clicked_col].piece
                        if piece.color==game.next_player:
                            board.calc_moves(piece, clicked_row, clicked_col)
                            dragger.save_initial(event.pos)
                            dragger.drag_piece(piece)
                            game.show_bg(screen)
                            game.show_last_move(screen)
                            game.show_moves(screen)
                            game.show_pieces(screen)
                        
               
                elif event.type == pygame.MOUSEMOTION:  # Mouse motion
                    motion_row=event.pos[1]//SQSIZE
                    motion_col=event.pos[0]//SQSIZE
                    game.set_hover(motion_row,motion_col)
                    if dragger.dragging:
                        dragger.update_mouse(event.pos)
                        game.show_bg(screen)
                        game.show_last_move(screen)
                        game.show_moves(screen)
                        game.show_pieces(screen)
                        game.show_hover(screen)
                        dragger.update_blit(screen)
               
                elif event.type == pygame.MOUSEBUTTONUP: #click_release
                    if dragger.dragging:
                        dragger.update_mouse(event.pos)
                        released_row=dragger.mouseY // SQSIZE
                        released_col=dragger.mouseX // SQSIZE

                    # create possible move
                        initial=Square(dragger.initial_row,dragger.initial_col)
                        final=Square(released_row,released_col)
                        move=Move(initial,final)
                        
                        if board.valid_move(dragger.piece,move):
                            captured=board.squares[released_row][released_col].has_piece()
                    
                            board.move(dragger.piece,move)
                            game.play_sound(captured)
                            game.show_bg(screen)
                            game.show_last_move(screen)
                            game.show_pieces(screen)
                            game.next_turn()
                    dragger.undrag_piece()
                    
                elif event.type==pygame.KEYDOWN:
                    if event.key==pygame.K_t:
                        game.change_theme()
                        
                    if event.key==pygame.K_r:
                        game.reset()
                        screen, game = self.screen, self.game
                        dragger, board = game.dragger, game.board
               
                elif event.type == pygame.QUIT:  # Quit application
                    running = False

            pygame.display.update()
        pygame.quit()

class Game:
    def __init__(self):
        self.next_player='white'
        self.hovered_sqr=None
        self.board = Board()
        self.dragger = Dragger()
        self.config=Config()

    def show_bg(self, surface):
        theme = self.config.theme
        for row in range(ROWS):
            for col in range(COLS):
                color = theme.bg.light if (row + col) % 2 == 0 else theme.bg.dark
                rect = (col * SQSIZE, row * SQSIZE, SQSIZE, SQSIZE)
                pygame.draw.rect(surface, color, rect)
                # row co-ordinates
                if col==0:
                    color=theme.bg.dark if row % 2==0 else theme.bg.light
                    lbl=self.config.font.render(str(ROWS-row),1,color)
                    lbl_pos=(5,5+row*SQSIZE)
                    surface.blit(lbl,lbl_pos)

                if row==7:
                    color=theme.bg.dark if (row+col) % 2==0 else theme.bg.light
                    lbl=self.config.font.render(Square.get_alphacol(col),1,color)
                    lbl_pos=(col*SQSIZE+SQSIZE-20,HEIGHT-20)
                    surface.blit(lbl,lbl_pos)

    def show_pieces(self, surface):
        for row in range(ROWS):
            for col in range(COLS):
                if self.board.squares[row][col].has_piece():
                    piece = self.board.squares[row][col].piece
                    
                    if piece is not self.dragger.piece:
                        piece.set_texture(size=80)
                        img = pygame.image.load(piece.texture)
                        img_center = col * SQSIZE + SQSIZE // 2, row * SQSIZE + SQSIZE // 2
                        piece.texture_rect = img.get_rect(center=img_center)
                        surface.blit(img, piece.texture_rect)

    def show_moves(self, surface):
        theme=self.config.theme
        if self.dragger.dragging:
            piece = self.dragger.piece
            for move in piece.moves:
                color = theme.moves.light if (move.final.row + move.final.col) % 2 == 0 else theme.moves.dark
                rect = (move.final.col * SQSIZE, move.final.row * SQSIZE, SQSIZE, SQSIZE)
                pygame.draw.rect(surface, color, rect)

                
                
    def show_last_move(self, surface):
        theme=self.config.theme
        if self.board.last_move:
            initial = self.board.last_move.initial
            final = self.board.last_move.final
            for pos in [initial, final]:
                color = theme.trace.light if (pos.row + pos.col) % 2 == 0 else theme.trace.dark
                rect = (pos.col * SQSIZE, pos.row * SQSIZE, SQSIZE, SQSIZE)
                pygame.draw.rect(surface, color, rect)

    def show_hover(self,surface):
        if self.hovered_sqr:
            color = (180, 180, 180) 
            rect = (self.hovered_sqr.col * SQSIZE, self.hovered_sqr.row * SQSIZE, SQSIZE, SQSIZE)
            pygame.draw.rect(surface, color, rect,width=3)
            

    def next_turn(self):
        self.next_player='white' if self.next_player=='black' else 'black'

    def set_hover(self,row,col):
        self.hovered_sqr=self.board.squares[row][col]

    def change_theme(self):
        self.config.change_theme()

    def play_sound(self,captured=False):
        if captured:
            self.config.capture_sound.play()
        else:
            self.config.move_sound.play()
   
    def reset(self):
        self.__init__()


class Square:
    ALPHACOLS={0:'a',1:'b',2:'c',3:'d',4:'e',5:'f',6:'g',7:'h'}
    def __init__(self, row, col, piece=None):
        self.row = row
        self.col = col
        self.piece = piece
        self.alphacol=self.ALPHACOLS[col]
        # return ALPHACOLS

    def __eq__(self,other):
        return self.row==other.row and self.col==other.col

    def has_piece(self):
        return self.piece is not None

    def isempty(self):
        return not self.has_piece()

    def has_team_piece(self, color):
        return self.has_piece() and self.piece.color == color
        
    def has_enemy_piece(self, color):
        return self.has_piece() and self.piece.color != color
        
    def isempty_or_enemy(self, color):
        return self.isempty() or self.has_enemy_piece(color)

    @staticmethod
    def in_range(*args):
        for arg in args:
            if arg < 0 or arg > 7:
                return False
        return True
    @staticmethod
    def get_alphacol(col):
        ALPHACOLS={0:'a',1:'b',2:'c',3:'d',4:'e',5:'f',6:'g',7:'h'}
        return ALPHACOLS[col]

class Board:
    def __init__(self):
        self.squares = [[Square(row, col) for col in range(COLS)] for row in range(ROWS)]
        self.last_move=None
        self._add_pieces('white')
        self._add_pieces('black')

    def move(self,piece,move):
        initial=move.initial
        final=move.final
                      # console board move update
        self.squares[initial.row][initial.col].piece=None
        self.squares[final.row][final.col].piece=piece
        
        # pawn_promotion

        if isinstance(piece,Pawn):
            self.check_promotion(piece,final)
            
        # king castling
        if isinstance(piece,King):
            if self.castling(initial,final):
                diff=final.col-initial.col
                rook = piece.left_rook if (diff<0) else piece.right_rook
                self.move(rook,rook.moves[-1])
        
        piece.moved=True
        piece.clear_moves()
        self.last_move=move

    def valid_move(self,piece,move):
        return move in piece.moves
        
    def check_promotion(self,piece,final):
        if final.row==0 or final.row==7:
            self.squares[final.row][final.col].piece=Queen(piece.color)
            
    def castling(self,initial,final):
        return abs(initial.col-final.col)==2

    def calc_moves(self, piece, row, col):
        pass
        
        def knight_moves():
            possible_moves = [
                (row-2, col+1), (row-1, col+2), (row+1, col+2), (row+2, col+1),
                (row+2, col-1), (row+1, col-2), (row-1, col-2), (row-2, col-1)
            ]
            for possible_move in possible_moves:
                possible_move_row, possible_move_col = possible_move
                if Square.in_range(possible_move_row, possible_move_col):
                    if self.squares[possible_move_row][possible_move_col].isempty_or_enemy(piece.color):
                        initial = Square(row, col)
                        final = Square(possible_move_row, possible_move_col)
                        move = Move(initial, final)
                        piece.add_move(move)

       
             
        def pawn_moves():
            steps = 1 if piece.moved else 2
    
    # Vertical moves
            for i in range(1, steps + 1):
                possible_move_row = row + piece.dir * i
                if Square.in_range(possible_move_row) and self.squares[possible_move_row][col].isempty():
            # Create the move
                    initial = Square(row, col)
                    final = Square(possible_move_row, col)
                    move = Move(initial, final)
                    piece.add_move(move)
                else:
                    break  # Stop if there's an obstacle

    # Diagonal moves for capturing an enemy piece
            possible_move_row = row + piece.dir
            possible_move_cols = [col - 1, col + 1]
            for possible_move_col in possible_move_cols:
                if Square.in_range(possible_move_row, possible_move_col):
                    if self.squares[possible_move_row][possible_move_col].has_enemy_piece(piece.color):
                        initial = Square(row, col)
                        final = Square(possible_move_row, possible_move_col)
                # Create a new move
                        move = Move(initial, final)
                        piece.add_move(move)

       
        
        def straightline_moves(incrs):
                for incr in incrs:
                    row_incr,col_incr=incr
                    possible_move_row = row + row_incr
                    possible_move_col = col + col_incr
                    while True:
                        if Square.in_range(possible_move_row, possible_move_col):
                            initial = Square(row, col)
                            final = Square(possible_move_row, possible_move_col)
                            move = Move(initial, final)
                            if self.squares[possible_move_row][possible_move_col].isempty():
                                        piece.add_move(move)
                            if self.squares[possible_move_row][possible_move_col].has_enemy_piece(piece.color):
                                         piece.add_move(move)
                                         break
                        else:break
                        possible_move_row=possible_move_row+row_incr
                        possible_move_col=possible_move_col+col_incr
                        
        def king_moves():
            adjs=[
                (row-1,col+0),#up
                (row-1,col+1),#up-right
                (row+0,col+1),#right
                (row+1,col+1),#down-right
                (row+1,col+0),#down
                (row+1,col-1),#down-left
                (row+0,col-1),#left
                (row-1,col-1),#up-left
                ]
            # normal moves
            for possible_move in adjs:
                    possible_move_row,possible_move_col=possible_move
                    if Square.in_range(possible_move_row,possible_move_col):
                        if self.squares[possible_move_row][possible_move_col].isempty_or_enemy(piece.color):
                            initial = Square(row, col)
                            final = Square(possible_move_row, possible_move_col)
                            move = Move(initial, final)
                            piece.add_move(move)
                            
            # casteling moves(queen , king)
            if not piece.moved:
                left_rook=self.squares[row][0].piece
                if isinstance(left_rook,Rook):
                    if not left_rook.moved:
                        for c in range(1,4):
                            if self.squares[row][c].has_piece():
                                break
                            if c==3:
                                piece.left_rook=left_rook
                                # rook move
                                initial=Sqaure(row,0)
                                final=Square(row,3)
                                move=Move(initial,final)
                                left_rook.add_move(move)
                                # king move
                                initial=Square(row,col)
                                final=Square(row,2)
                                move=Move(initial,final)
                                piece.add_move(move)
        
                # king castling
                right_rook=self.squares[row][7].piece
                if isinstance(right_rook,Rook):
                    if not right_rook.moved:
                        for c in range(5,7):
                            if self.squares[row][c].has_piece():
                                break
                            if c==3:
                                piece.right_rook=right_rook
                                # rook move
                                initial=Sqaure(row,7)
                                final=Square(row,5)
                                move=Move(initial,final)
                                right_rook.add_move(move)
                                # king move
                                initial=Sqaure(row,col)
                                final=Square(row,6)
                                move=Move(initial,final)
                                piece.add_move(move)

        

       
        if piece.name == 'pawn':
            pawn_moves()

        elif piece.name == 'knight':
            knight_moves()

        elif piece.name == 'rook':
            straightline_moves([
                (-1,0),#up
                (0,1),#right
                (1,0),#down
                (0,-1)#left
            ])

        elif piece.name == 'bishop':
            straightline_moves([(-1,1),#up right
                                (-1,-1),#up left
                                (1,1),#down right
                                (1,-1)#down left
                               ])

        elif piece.name == 'queen':
            straightline_moves([(-1,1),#up right
                                (-1,-1),#up left
                                (1,1),#down right
                                (1,-1),#down left
                                (-1,0),#up
                                (0,1),#right
                                (1,0),#down
                                (0,-1)#left
                               ])

        elif piece.name == 'king':
            king_moves()

    def _add_pieces(self, color):
        row_pawn, row_other = (6, 7) if color == 'white' else (1, 0)
        
        # Add pawns
        for col in range(COLS):
            self.squares[row_pawn][col] = Square(row_pawn, col, Pawn(color))
            # self.squares[5][0] = Square(5, 0, Pawn(color))
       
        # Add knights
        self.squares[row_other][1] = Square(row_other, 1, Knight(color))
        self.squares[row_other][6] = Square(row_other, 6, Knight(color))
        # self.squares[4][4]=Square(4,4,Knight(color))

        # Add bishops
        self.squares[row_other][2] = Square(row_other, 2, Bishop(color))
        self.squares[row_other][5] = Square(row_other, 5, Bishop(color))
        # self.squares[5][7] = Square(5, 7, Bishop(color))

        # Add rooks
        self.squares[row_other][0] = Square(row_other, 0, Rook(color))
        self.squares[row_other][7] = Square(row_other, 7, Rook(color))
        # self.squares[4][4] = Square(4, 4, Rook(color))

        # Add queen
        self.squares[row_other][3] = Square(row_other, 3, Queen(color))
        # self.squares[3][3] = Square(3, 3, Queen(color))

        # Add king
        self.squares[row_other][4] = Square(row_other, 4, King(color))
        # self.squares[2][4] = Square(2, 4, King(color))


class Piece:
    def __init__(self, name, color, value, texture=None, texture_rect=None):
        self.name = name
        self.color = color
        value_sign = 1 if color == 'white' else -1
        self.value = value * value_sign
        self.moves = []
        self.moved = False
        self.texture = texture
        self.texture_rect = texture_rect
        self.set_texture()

    def set_texture(self, size=80):
        self.texture = os.path.join(f'C:/Users/Devika/{self.color}_{self.name}.png')

    def add_move(self, move):
        self.moves.append(move)

    def clear_moves(self):
        self.moves=[]

class Pawn(Piece):
    def __init__(self, color):
        self.dir = -1 if color == 'white' else 1
        super().__init__('pawn', color, 1.0)

class Knight(Piece):
    def __init__(self, color):
        super().__init__('knight', color, 3.0)

class Bishop(Piece):
    def __init__(self, color):
        super().__init__('bishop', color, 3.001)

class Rook(Piece):
    def __init__(self, color):
        super().__init__('rook', color, 5.0)

class Queen(Piece):
    def __init__(self, color):
        super().__init__('queen', color, 9.0)

class King(Piece):
    def __init__(self, color):
        self.left_rook=None
        self.right_rook=None
        super().__init__('king', color, 10000.0)

# Dragger class for dragging pieces
class Dragger:
    def __init__(self):
        self.piece = None
        self.dragging = False
        self.mouseX = 0
        self.mouseY = 0
        self.initial_row = 0
        self.initial_col = 0

    def update_blit(self, surface):
        if self.piece:
            self.piece.set_texture(size=128)  # Texture of selected piece
            texture = self.piece.texture
            img = pygame.image.load(texture)  # Image size and texture
            img_center = (self.mouseX, self.mouseY)
            self.piece.texture_rect = img.get_rect(center=img_center)
            surface.blit(img, self.piece.texture_rect)

    def update_mouse(self, pos):
        self.mouseX, self.mouseY = pos

    def save_initial(self, pos):
        self.initial_row = pos[1] // SQSIZE
        self.initial_col = pos[0] // SQSIZE

    def drag_piece(self, piece):
        self.piece = piece
        self.dragging = True

    def undrag_piece(self):
        self.piece = None
        self.dragging = False

class Move:
    def __init__(self, initial, final):
        self.initial = initial
        self.final = final
    def __str__(self):
        s=''
        s+=f'({self.initial_col},{self.initial_row})'
        s+=f'->({self.final_col},{self.final_row})'
        return s
    def __eq__(self,other):
        return self.initial==other.initial and self.final==other.final


class Config:
    def __init__(self):
        self.themes = []
        self._add_themes()
        self.idx = 0
        self.theme = self.themes[self.idx]
        self.font=pygame.font.SysFont("monospace",18,bold=True)
        # Load sounds
        self.move_sound = Sound(os.path.join('C:/Users/Devika/move.wav'))
        self.capture_sound = Sound(os.path.join('C:/Users/Devika/capture.wav'))

    def change_theme(self):
        self.idx += 1
        self.idx %= len(self.themes)
        self.theme = self.themes[self.idx]

    
    def _add_themes(self):
    # Add different themes for the board and pieces
        self.themes.append(Theme(
            light=(234, 235, 200), 
            dark=(119, 154, 88),
            moves_light=(244, 247, 116), 
            moves_dark=(172, 195, 51),
            trace_light='#C86464', 
            trace_dark='#C84646'
        ))

        self.themes.append(Theme(
            light=(235, 209, 166), 
            dark=(165, 117, 80),
            moves_light=(245, 234, 100), 
            moves_dark=(209, 185, 59),
            trace_light='#C86464', 
            trace_dark='#C84646'
        ))

    # Adding Gray Theme
        self.themes.append(Theme(
            light=(120, 119, 118), 
            dark=(86, 85, 84),
            moves_light=(99, 126, 143), 
            moves_dark=(82, 102, 128),
            trace_light='#C86464', 
            trace_dark='#C84646'
        ))

    # Adding Blue Theme
        self.themes.append(Theme(
            light=(229, 228, 200), 
            dark=(60, 95, 135),
            moves_light=(123, 187, 227), 
            moves_dark=(43, 119, 191),
            trace_light='#C86464', 
            trace_dark='#C84646'
        ))

 


class Sound:
    def __init__(self, path):
        self.path = path
        self.sound = pygame.mixer.Sound(path)

    def play(self):
        self.sound.play()


class Theme:
    def __init__(self, light, dark, moves_light, moves_dark, trace_light, trace_dark):
        self.bg = BgColors(light, dark)
        self.moves = MovesColors(moves_light, moves_dark)
        self.trace = TraceColors(trace_light, trace_dark)

class BgColors:
    def __init__(self, light, dark):
        self.light = light
        self.dark = dark

class MovesColors:
    def __init__(self, light, dark):
        self.light = light
        self.dark = dark

class TraceColors:
    def __init__(self, light, dark):
        self.light = light
        self.dark = dark

                 


# Initialize and start the main loop
if __name__ == "__main__":
    main = Main()
    main.mainloop()




pygame 2.6.0 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html
